<a href="https://colab.research.google.com/github/antcho1024/-RANSAC_linefitting/blob/main/%08RANSAC_linefitting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
import cv2 as cv
import numpy as np
from google.colab.patches import cv_imshow

img = cv.imread('/content/gdrive/MyDrive/MachineVision/lanes.bmp',cv.IMREAD_GRAYSCALE)
img1 = cv.imread('/content/gdrive/MyDrive/MachineVision/lanes.bmp',cv.IMREAD_GRAYSCALE)

In [ ]:
Ix = cv.Sobel(img,ddepth=cv.CV_64F,dx=1,dy=0,ksize=3)
Iy = cv.Sobel(img,ddepth=cv.CV_64F,dx=0,dy=1,ksize=3)
mag = np.sqrt(np.square(Ix) + np.square(Iy))
ori = np.arctan2(Iy,Ix) * 180/np.pi

In [ ]:
# display 하기 위해 값의 범위 0-255로 변경
Ix_ = (Ix-Ix.min()) / (Ix.max()-Ix.min()) * 255
Iy_ = (Iy-Iy.min()) / (Iy.max()-Iy.min()) * 255
mag_ = (mag-mag.min()) / (mag.max()-mag.min()) * 255 
ori_ = (ori-ori.min()) / (ori.max()-ori.min()) * 255

In [ ]:
'''cv_imshow(img)
cv_imshow(Ix_) 
cv_imshow(Iy_) 
cv_imshow(mag_) 
cv_imshow(ori_)
'''

In [ ]:
result_left = np.zeros(img.shape)
id_L = np.where((mag>100) & (ori>30) & (ori<60)) 
result_left[id_L] = 255

#cv_imshow(result_left)

In [ ]:
result_right = np.zeros(img.shape)
id_R = np.where((mag>100) & (ori>-60) & (ori<-30)) 
result_right[id_R] = 255

#cv_imshow(result_right)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def f(x,a,b): 
    return a*x+b


def ransac_line_fiting(x,y,r,t):
    iter = np.round(np.log(1-0.999) / np.log(1-(1-r)**2) + 1) 
    num_max = 0
    for i in np.arange(iter):
        id = np.random.permutation(len(x))
        xs = x[id[:2]]
        ys = y[id[:2]]
        A = np.vstack([xs, np.ones(len(xs))]).T
        ab = np.dot(np.linalg.inv(np.dot(A.T, A)), np.dot(A.T, ys)) 
        dist = np.abs(ab[0]*x-y+ab[1])/np.sqrt(ab[0]**2+1) 
        numInliers = sum(dist < t)
        if numInliers > num_max: 
            ab_max = ab
            num_max = numInliers
    return ab_max, num_max

In [ ]:
#좌측차선 Line fitting
abno_L, max = ransac_line_fiting(id_L[1],id_L[0],0.1,2)
abno_L

In [ ]:
id_L[1]

In [ ]:
img_tmp_L = img
y=f(100,abno_L[0],abno_L[1])
y1=f(270,abno_L[0],abno_L[1])
result_L=cv.line(img_tmp_L,(100,int(y)),(270,int(y1)),(0,255,255),5)
cv_imshow(result_L)

In [ ]:
#우측차선 Line fitting
abno_R, max = ransac_line_fiting(id_R[1],id_R[0],0.1,2)
abno_R

In [ ]:
id_R[1]

In [ ]:
img_tmp_R = img1
y=f(385,abno_R[0],abno_R[1])
y1=f(570,abno_R[0],abno_R[1])
result_R=cv.line(img_tmp_R,(385,int(y)),(570,int(y1)),(0,255,0),5)
cv_imshow(result_R)